In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,013 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:14 http://ppa.launchpad.net/graphi

In [3]:
!pip install --upgrade --no-cache-dir gdown
!gdown https://drive.google.com/u/0/uc?id=1kCXnIeoPT6p9kS_ANJ0mmpxlfDwK1yio&export=download
!unzip DatasetToCompleteTheSixSparkExercises.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Downloading...
From: https://drive.google.com/u/0/uc?id=1kCXnIeoPT6p9kS_ANJ0mmpxlfDwK1yio
To: /content/DatasetToCompleteTheSixSparkExercises.zip
100% 6.43G/6.43G [00:54<00:00, 119MB/s]
Archive:  DatasetToCompleteTheSixSparkExercises.zip
   creating: products_parquet/
  inflating: products_parquet/.part-00000-0f978a44-491b-411b-9bf5-266bd9d4e836-c000.snappy.parquet.crc  
  inflating: products_parquet/.part-00001-0f978a44-491b-411b-9bf5-266bd9d4e836-c000.snappy.parquet.crc  
  inflating: products_parquet/.part-00002-0f978a44-491b-411b-9bf5-266bd9d4e836-c000.snappy.parquet.crc  
  inflating: products_parquet/.part-00003-0f978a44-491b-411b-9bf5-266bd9d4e836-c000.snappy.parquet.crc  
  inflating: products_parquet/.part-00004-0f978a44-491b-411b-9bf5

In [52]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

spark = SparkSession.builder.config('spark.ui.port', '4050')\
                            .config("spark.driver.memory","8g")\
                            .config("spark.executor.memory","8g")\
                            .getOrCreate()

In [4]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2023-03-06 00:17:31--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  86.7MB/s    in 0.2s    

2023-03-06 00:17:31 (86.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [39]:
!./ngrok authtoken 2MZy0Gkfwx8h9nNkOVbG0bAcdR4_7ig9Gpx5QsUmHPpeVQ7Rw

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [40]:
get_ipython().system_raw('./ngrok http 4050 &')
#Aguarde 10s

In [41]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://0807-35-188-242-127.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://0807-35-188-242-127.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [53]:
sales_dtf = spark.read.parquet("sales_parquet")
#products_dtf = spark.read.parquet("products_parquet")
#sellers_dtf = spark.read.parquet("sellers_parquet")

sales_dtf.cache().count()
#sellers_dtf.count()
#products_dtf.count()

#sales_dtf.show(5)
#sellers_dtf.show(5)
#products_dtf.show(5)

20000040

In [54]:
sales_dtf.rdd.getNumPartitions()

83

In [55]:
products_dtf = spark.read.parquet("products_parquet")

In [58]:
joined = sales_dtf.join(products_dtf,on="product_id")

In [59]:
joined.show()

+----------+--------+---------+----------+---------------+--------------------+----------------+-----+
|product_id|order_id|seller_id|      date|num_pieces_sold|       bill_raw_text|    product_name|price|
+----------+--------+---------+----------+---------------+--------------------+----------------+-----+
|  10005243|12478308|        6|2020-07-04|             98|qfvpgiscflyjxphcq...|product_10005243|   44|
|  10023464| 8996776|        9|2020-07-03|             59|jjbyqkzcimBfoehbv...|product_10023464|   19|
|  10050363|10476976|        6|2020-07-03|             18|xqhlvkpxtzrfdadry...|product_10050363|   98|
|  10089524| 5977582|        2|2020-07-01|             53|jchvhzbeaicqitpvx...|product_10089524|  100|
|  10122266| 1482892|        2|2020-07-07|             25|kezhpglnqigaqwrss...|product_10122266|   89|
|  10134574| 2987749|        3|2020-07-03|             28|bmtlxvobfmerqlckf...|product_10134574|  138|
|  10150439|15982399|        4|2020-07-01|             72|rvgiffufcxceotp